In [ ]:
import cv2
import numpy as np
import math
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, frame = cap.read()
    cv2.rectangle(frame,(300,300),(100,100),(0,255,0),0)
    crop_img = frame[100:300, 100:300]
    frame=cv2.flip(frame,1) 
    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    value = (35, 35)
    blurred = cv2.GaussianBlur(grey, value, 0)
    _, thresh1 = cv2.threshold(blurred, 127, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    cv2.imshow('Thresholded', thresh1)
    contours, hierarchy = cv2.findContours(thresh1.copy(),cv2.RETR_TREE, \
            cv2.CHAIN_APPROX_NONE)
    max_area = -1
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i
    cnt=contours[ci]
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(crop_img,(x,y),(x+w,y+h),(0,0,255),0)
    hull = cv2.convexHull(cnt)
    areahull = cv2.contourArea(hull)
    areacnt = cv2.contourArea(cnt)
    arearatio=((areahull-areacnt)/areacnt)*100
    drawing = np.zeros(crop_img.shape,np.uint8)
    cv2.drawContours(drawing,[cnt],0,(0,255,0),0)
    cv2.drawContours(drawing,[hull],0,(0,0,255),0)
    hull = cv2.convexHull(cnt,returnPoints = False)
    defects = cv2.convexityDefects(cnt,hull)
    l = 0
    cv2.drawContours(thresh1, contours, -1, (0,255,0), 3)
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
        if angle <= 90 and d>30:
                l += 1
                cv2.circle(crop_img, far, 3, [255,0,0], -1)
            
            #draw lines around hand
        cv2.line(crop_img,start, end, [0,255,0], 2)
            
        l+=1     
        
        #print corresponding gestures which are in their ranges
        font = cv2.FONT_HERSHEY_SIMPLEX
        if l==1:
            if areacnt<2000:
                cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            else:
                if arearatio<12:
                    cv2.putText(frame,' 0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif arearatio<23:
                    cv2.putText(frame,' 1',(30,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                   
                else:
                    cv2.putText(frame,' ',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        elif l==2 and arearatio>25 and arearatio<36:
            #if arearatio<24 and arearatio>13:
                cv2.putText(frame,'2',(50,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        elif l==3 and arearatio>36 and arearatio<45:
           # if arearatio<70 and arearatio>46:
                    cv2.putText(frame,'3',(80,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            #else:
             #   cv2.putText(frame,'ok',(40,40), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    
        elif l==4 and arearatio>46 and arearatio<60:
          #  if arearatio<80 and arearatio>65:
                cv2.putText(frame,'4',(130,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l==5 and arearatio>60 and arearatio<75:
            cv2.putText(frame,'5',(180,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l==6:
            cv2.putText(frame,' ',(100,70), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        else :
            cv2.putText(frame,' ',(70,80), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
    #cv2.imshow('drawing', drawing)
    #cv2.imshow('end', crop_img)
    cv2.imshow('Gesture', frame)
    all_img = np.hstack((drawing, crop_img))
    cv2.imshow('Contours', all_img)
    k = cv2.waitKey(30)
    if k == 27:
        break
cv2.destroyAllWindows()
cap.release()